In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import InceptionResNetV2

from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers, Sequential
from keras.layers.normalization import BatchNormalization
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
batch_size = 64

In [4]:
# rescale normalizes the data for the images.
# Data is split into train and validation sets
image_data_gen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    validation_split=0.2)

train_it = image_data_gen.flow_from_directory(directory='/content/drive/MyDrive/tb2/TB_Chest_Radiography_Database',
                                              target_size=(299, 299),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='training'
                                             )

val_it = image_data_gen.flow_from_directory(directory='/content/drive/MyDrive/tb2/TB_Chest_Radiography_Database',
                                              target_size=(299, 299),
                                              color_mode='rgb',
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='validation'
                                             )

Found 4264 images belonging to 2 classes.
Found 1066 images belonging to 2 classes.


In [5]:
dnn201 = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    input_tensor=None,
    input_shape=(299,299,3),
    weights = 'imagenet'
)

dnn201.trainable= False

global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
predictions = Dense(2, activation='softmax')

model = Sequential([
    dnn201,
    global_avg_layer,
    
    BatchNormalization(),
    Dropout(0.5),
    
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    
    predictions
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

219062272/219055592 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 8, 8, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
batch_normalization_203 (Bat (None, 1536)              6144      
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1573888   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_______________________________________________________

In [6]:
epochs = 10
model_hist = model.fit(train_it,
#                       validation_data=val_it)
                       epochs=epochs, )
model.save('InceptionResNetV2')

Epoch 1/10
67/67 [==============================] - 1451s 21s/step - loss: 0.5146 - accuracy: 0.8344
Epoch 2/10
67/67 [==============================] - 123s 2s/step - loss: 0.1683 - accuracy: 0.9415
Epoch 3/10
67/67 [==============================] - 124s 2s/step - loss: 0.1869 - accuracy: 0.9335
Epoch 4/10
67/67 [==============================] - 124s 2s/step - loss: 0.1470 - accuracy: 0.9465
Epoch 5/10
67/67 [==============================] - 123s 2s/step - loss: 0.1183 - accuracy: 0.9548
Epoch 6/10
67/67 [==============================] - 124s 2s/step - loss: 0.1264 - accuracy: 0.9591
Epoch 7/10
67/67 [==============================] - 124s 2s/step - loss: 0.1147 - accuracy: 0.9590
Epoch 8/10
67/67 [==============================] - 125s 2s/step - loss: 0.1054 - accuracy: 0.9596
Epoch 9/10
67/67 [==============================] - 124s 2s/step - loss: 0.1092 - accuracy: 0.9600
Epoch 10/10
67/67 [==============================] - 123s 2s/step - loss: 0.0978 - accuracy: 0.9626
INFO:te

In [7]:
results = model.evaluate(val_it)

17/17 [==============================] - 324s 20s/step - loss: 0.0747 - accuracy: 0.9737
